# Numerical Analysis: Orthogonality and EV

In [3]:
import numpy.linalg as la
import numpy.random as rn
import numpy as np

In [4]:
def LU(A):
    m,n = A.shape
    U = A.copy()
    L = np.mat(np.identity(m))
    P = L.copy()
    
    for i in range(m):
        # find max element in the col i
        maxEc = abs(U[i,i])
        maxRow = i
        for k in range(i+1,m):
            if abs(U[k,i]) > maxEc:
                maxEc = U[k,i]
                maxRow = k
        
        #swap maximum row with current row
        U[[i, maxRow],:] = U[[maxRow,i],:]
        P[[i, maxRow],:] = P[[maxRow,i],:]
        L[[i, maxRow],:] = L[[maxRow,i],:]
        L[:,[i, maxRow]] = L[:,[maxRow,i]] #maintain low triangular form
        
        for k in range(i+1,m):
            c = -float(U[k,i])/U[i,i]
            #delete k row by i
            U[k,i:] = U[k,i:] + c*U[i,i:]
            #delete i col by k
            L[k:,i] = L[k:,i] - c*L[k:,k]
    
    return L,U,P

def Forward(L,b):
    m,n = L.shape
    x = b.copy()
    for i in range(m):
        for k in range(i):
            x[i] -= L[i,k]*x[k] # in the first step i = 0 will not occur 
        x[i] = float(x[i])/L[i,i] # i, i is the pivot
    return x

def Backward(U,b):
    m,n = U.shape
    x = b.copy()
    for i in range(m-1,-1,-1):
        for k in range(m-1,i,-1):
            x[i] -= U[i,k]*x[k]
        x[i] = float(x[i])/U[i,i]
    return x

## Inverse Power Method

$x_{n+1} = (A-\mu I)^{-1}x_n \to (A-\mu I)x_{n+1}=x_n$

E.val: $\dfrac{1}{\lambda - \mu}$ max val when $\lambda$ is close to $\mu$

In [3]:
#find the mu-nearest eigenvalue
def inv_power(A, mu, tol=10**(-12), Maxitr=10000):
    m,n = A.shape
    B = A-mu*np.identity(m)
    x0 = np.mat(rn.randn(m,1))
    x0 = x0/la.norm(x0) #normalized
    L,U,P = LU(B)
    bp = P*x0
    y = Forward(L,bp) #solve y
    x1 = Backward(U,y) #solve x
    x1 = x1/la.norm(x1) #normalized
    i = 0
    while (la.norm(x1-x0))>tol and abs(la.norm(x1-x0)-2)>tol and i<Maxitr:
        # la.norm(x1-x0)-2 sometime there is close but diff direction
        i+=1
        x0 = x1
        bp = P*x0
        y = Forward(L,bp) #solve y
        x1 = Backward(U,y) #solve x
        x1 = x1/la.norm(x1) #normalized
        if i==Maxitr:
            print(la.norm(x1-x0))
            print("Warning: not accurate")
    print("Compute",i,"iteration to obtain the result")
    L = (A*x1).T*x1/(x1.T*x1) #e.val
    return x1,L

In [6]:
A = np.mat(np.round(rn.randn(5,5)*10))
A = A.T*A #make sure there exist ev
A

matrix([[ 586.,  182.,   26., -357.,  -38.],
        [ 182.,  463.,   64., -167.,  152.],
        [  26.,   64.,  133.,   76.,   14.],
        [-357., -167.,   76., 1062.,  138.],
        [ -38.,  152.,   14.,  138.,  112.]])

In [7]:
a = la.eig(A)
print(a[0])

[1325.82140692  558.31434728  344.16851806  115.40180799   12.29391975]


In [9]:
x1, L = inv_power(A,2000)
L

Compute 19 iteration to obtain the result


matrix([[1325.82140692]])

In [10]:
x1, L = inv_power(A,(1325+558)/2)
L

Compute 5712 iteration to obtain the result


matrix([[558.31434728]])

if mu is between 2 eigen val, there can be very slow

## Power Method

find max$\lambda$

theorem: A is nxn diagonalizable, exist x0 != 0, s.t.

$Ax_0, A^2x_0, ..., A^nx_0$

converge to max e.vector

proof:

$Ax_0$ can be expressed by linear combination of of e.vec

$Ax_0=\sum c_iAx_i = \sum c_i \lambda _ix_i$

$A^kx_0=\sum c_i \lambda ^k x_i = \lambda _1^k [c_1x_1+c_2(\dfrac{\lambda _2}{\lambda _1})^kx_2+...+c_n(\dfrac{\lambda _n}{\lambda _1})^kx_n]$

WLOG, lambda 1 is the largest


In [12]:
def power(A, tol=10**(-12), Maxitr=10000):
    m,n = A.shape
    x0 = np.mat(rn.randn(m,1))
    x0 = x0/la.norm(x0) #normalized
    x1 = A*x0
    x1 = x1/la.norm(x1) #normalized
    i = 1
    while (la.norm(x1-x0))>tol and i<Maxitr:
        i+=1
        x0 = x1
        x1 = A*x0
        x1 = x1/la.norm(x1)
    print("Compute",i,"iteration to obtain the result")
    L = (A*x1).T*x1/(x1.T*x1) #e.val
    return x1,L

In [13]:
x1,L=power(A)
L

Compute 33 iteration to obtain the result


matrix([[1325.82140692]])

## Orthogonalization of Vectors

In [14]:
a = np.mat(rn.randn(3,1))
b = np.mat(rn.randn(3,1))

t = float(a.T*b)
t

-0.24274357768078647

In [15]:
a1 = a-t*b/la.norm(b)**2
a2 = a-a1

In [16]:
float(a1.T*b) #pendicular vector

-2.220446049250313e-16

The case of unit vector

In [18]:
u = a/la.norm(a)
v = b/la.norm(b)
c = float(u.T*v)
w = u-c*v
w = w/la.norm(w)
float(w.T*v)

5.551115123125783e-17

In [19]:
la.norm(w)

0.9999999999999999

## Gram-Schmidt Processing

q1:

$q_1 = \frac{x_1}{\left \| x_1 \right \|}$

q2:

$q_{2}^{*} = (I-q_1q_1^T)x_2,\ q_2 = \frac{q_{2}^{*}}{\left \| q_{2}^{*} \right \|}$

qi:

$q_{i}^{*} = (I-\sum_{j=1}^{i-1}q_jq_j^T)x_i,\ q_i = \frac{q_{i}^{*}}{\left \| q_{i}^{*} \right \|}$


In [5]:
def GS(A, tol=10**(-12)): #innerproduct def is 0 (represented by tol)
    m,n = A.shape
    Idx = []
    Q = np.mat(np.zeros((m,n)))
    Q[:,0] = A[:,0]/la.norm(A[:,0]) #first col
    for i in range(1,n):
        Q[:,i] = A[:,i].copy()
        #substract prior component of vector
        for j in range(i):
            Q[:,i] = Q[:,i] - Q[:,j]*(float(Q[:,j].T*Q[:,i])) 
        #if it is small enough dont do that
        if la.norm(Q[:,i])>tol:
            Q[:,i] = Q[:,i]/la.norm(Q[:,i])
        else:
            Idx.append(i)
    return Q, Idx

In [33]:
A = np.mat(rn.randn(4,6))
A

matrix([[ 0.46703609, -1.74634285,  0.08995412,  0.11939538, -0.90393909,
          0.11875354],
        [ 0.24062243, -0.94535212,  1.32090032, -0.32472793, -0.02607335,
         -1.23874079],
        [ 0.52844142, -0.11521273, -0.16309172, -1.21552488, -0.01222304,
          1.27182291],
        [-0.33724753, -0.86581688,  1.01085863, -0.17217811,  1.22388574,
         -0.45130582]])

In [34]:
Q, Idx = GS(A)

In [35]:
Q[:,:4]

matrix([[ 0.57099823, -0.61147663, -0.52792475,  0.14612603],
        [ 0.29418493, -0.33868797,  0.81313618,  0.37088441],
        [ 0.64607236,  0.27276511,  0.19289328, -0.68628119],
        [-0.41231875, -0.66104916,  0.15131819, -0.60836672]])

In [36]:
Q[:,0].T*Q[:,1]

matrix([[0.]])

In [37]:
Q[:,4:]

matrix([[ 8.32667268e-17, -8.32667268e-17],
        [-1.11022302e-16,  5.55111512e-17],
        [ 2.22044605e-16,  4.44089210e-16],
        [-2.22044605e-16, -3.33066907e-16]])

In [38]:
Q[:,0].T*Q[:,5]

matrix([[3.93028888e-16]])

In [39]:
#false GS
def FGS(A, tol=10**(-12)): #innerproduct def is 0 (represented by tol)
    m,n = A.shape
    Q = np.mat(np.zeros((m,n)))
    Q[:,0] = A[:,0]/la.norm(A[:,0]) #first col
    for i in range(1,n):
        Q[:,i] = A[:,i].copy()
        #substract prior component of vector
        for j in range(i):
            Q[:,i] = Q[:,i] - Q[:,j]*(float(Q[:,j].T*Q[:,i])) 
        Q[:,i] = Q[:,i]/la.norm(Q[:,i]) #false
    return Q

In [40]:
U = FGS(A)

In [42]:
U[:,4:]

matrix([[ 0.24253563, -0.61221303],
        [-0.32338083,  0.64282368],
        [ 0.64676167,  0.44895622],
        [-0.64676167, -0.1020355 ]])

In [43]:
U[:,0].T*U[:,5] #error not equal to 0

matrix([[0.17166584]])

## QR Decomposition

$\begin{bmatrix}
|&|&|\\ 
x_1&x_2&x_3\\ 
|&|&| 
\end{bmatrix} 
= 
\begin{bmatrix}
|&|&|\\ 
q_1&q_2&q_3\\ 
|&|&| 
\end{bmatrix} 
= 
\begin{bmatrix}
r_{11}&r_{12}&r_{13}\\ 
0&r_{22}&r_{23}\\ 
0&0 &r_{33} 
\end{bmatrix}
$
x1:

$ x_1 = r_{11}q_1 \Rightarrow r_{11} = \left \| x_1 \right \|$

x2:

$x_2 = r_{12}q_1+r_{22}q_2$

$q_1^Tx_2 = r_{12}q_1^Tq_1+r_{22}q_1^Tq_2 = r_{12}$

$r_{22}q_2 = x_2-q_1^Tx_2q_1 = q^*_2 \Rightarrow r_{22}=\left \|q^*_2  \right \|$

x3:

$q_1^Tx_3 = r_{13}$

$q_2^Tx_3 = r_{23}$

$r_{33}=\left \|q^*_3  \right \|$

in sum

$A=QR,\ R[r_{ij}]$

$r_{ij} = \left\{\begin{matrix}
q_i^TA^{(j)},\ i<j\\ 
\left \| q^*_i \right \|,\ i=j,\ q^*_i=(I-\sum_{k=1}^{i-1}q_kq_k^T)A^{(i)}\\ 
0,\ i>j
\end{matrix}\right.$

In [44]:
def QR(A, tol=10**(-14)):
    m,n = A.shape
    p = min(m,n)
    #there are p orthonormal vectors
    if m>=n:
        Q = np.mat(np.zeros((m,n)))
        R = np.mat(np.zeros((n,n)))
    else:
        Q = np.mat(np.zeros((m,m)))
        R = np.mat(np.zeros((m,n)))
    #record the rank of A
    r = 0
    #record the index of col
    i = 0
    while r<p and i<n: #full rank then stop
        Q[:,r] = A[:,i].copy()
        for j in range(r):
            #not diagonal part
            R[j,i] = float(Q[:,j].T*A[:,i])
            #GS decomposition
            Q[:,r] = Q[:,r] - R[j,i]*Q[:,j]
        if abs(la.norm(Q[:,r]))>tol: #find out a independent vector
            #diagonal part
            R[r,i] = la.norm(Q[:,r]) #maybe some initial col is not independent, norm could be not on diagonol
            #normalization
            Q[:,r] = Q[:,r]/R[r,i]
            r+=1
        i+=1
    R[:,m:] = Q.T*A[:,m:]
    
    return Q, R

In [45]:
A = np.mat(rn.randn(3,4))
Q,R = QR(A)

In [46]:
Q

matrix([[ 0.11472379, -0.96010936, -0.25500679],
        [ 0.09413804,  0.26605382, -0.95935051],
        [ 0.98892693,  0.08605449,  0.12090552]])

In [47]:
R

matrix([[ 1.73410591,  0.38720716, -0.28858312,  0.75113821],
        [ 0.        ,  1.7050537 , -0.30279115,  0.95234063],
        [ 0.        ,  0.        ,  0.78509429,  1.49014946]])

In [48]:
Q*R

matrix([[ 0.19894321, -1.59261614,  0.05740089, -1.20817596],
        [ 0.16324533,  0.49008697, -0.86090599, -1.1054911 ],
        [ 1.71490404,  0.52964711, -0.21652193,  1.00494128]])

In [49]:
A

matrix([[ 0.19894321, -1.59261614,  0.05740089, -1.20817596],
        [ 0.16324533,  0.49008697, -0.86090599, -1.1054911 ],
        [ 1.71490404,  0.52964711, -0.21652193,  1.00494128]])

In [50]:
la.norm(Q*R-A)

8.401507749314535e-16

In [53]:
#0,1,2 col is dependet
A[:,0] = A[:,1]
A[:,2] = A[:,1]
Q,R = QR(A)

In [54]:
Q

matrix([[-0.91086419, -0.13348462,  0.        ],
        [ 0.28029521, -0.8946213 ,  0.        ],
        [ 0.30292082,  0.42642067,  0.        ]])

In [55]:
R

matrix([[1.74846718, 1.74846718, 1.74846718, 1.095038  ],
        [0.        , 0.        , 0.        , 1.57879653],
        [0.        , 0.        , 0.        , 0.        ]])